### ISE 535 Final Project  - Exotic Option Price Simulator
#### Jinye Cai

In [1]:
from tkinter import *
from tkcalendar import *
import webbrowser
import numpy as np
from pandas_datareader import data
import datetime as dt
from dateutil.relativedelta import relativedelta

# Initialize Tiingo API
from tiingo import TiingoClient
config = {}
# To reuse the same HTTP Session across API calls (and have better performance), include a session key.
config['session'] = True
# If you don't have your API key as an environment variable,
# pass it in via a configuration dictionary.
config['api_key'] = "d8643081eae18b2bfb29e456b43e34f1dd8ebcbe"
# Initialize
client = TiingoClient(config)

start_date = dt.date.today() - relativedelta(years = 5)
today = dt.date.today()


ws = Tk()
ws.title('Exotic Option Price Simulator')
ws.geometry('800x550')
#ws.config(bg='#51a0cf')
        

# Get stock price and dividend
def stock_info():
    ticker = comp.get()
    try:
        quo = data.get_quote_yahoo(ticker)
        S = quo['price'][0]
        try:
            div = quo['trailingAnnualDividendYield'][0]
        except: div = 0
    except: 
        S = 'N/A'
        div = 0
    return S,div

def show_S():
    resultlabel=Label(ws,text=' '*50).grid(row=4,column=0)
    ticker = comp.get()
    S = stock_info()[0]
    resultlabel = Label(ws, text = f'{ticker.upper()} : {S}')
    return resultlabel.grid(row = 4,column = 0)

def show_div():
    resultlabel2=Label(ws,text=' '*50).grid(row=4,column=1)
    ticker = comp.get()
    div = stock_info()[1]
    resultlabel2 = Label(ws, text = f'{ticker.upper()} : {round(div*100,4)}%')
    return resultlabel2.grid(row = 4,column = 1)

# get 5 year volatility of the specified company
def comp_sigma():
    ticker = comp.get()
    price_history = client.get_dataframe([ticker],
                                      frequency='monthly',
                                      metric_name='adjClose',
                                      startDate=start_date,
                                      endDate=today)
    log_return = np.log(price_history[ticker]/price_history[ticker].shift(1))
    SIG = np.std(log_return)*np.sqrt(12)
    return SIG

def show_sigma():
    resultlabel3=Label(ws,text=' '*50).grid(row=4,column=2)
    ticker = comp.get()
    SIG = comp_sigma()
    resultlabel3 = Label(ws, text = f'{ticker.upper()} : {round(SIG*100,4)}%')
    return resultlabel3.grid(row = 4,column = 2)

# Get time to marturity
def get_T():
    maturity = cal.get_date()
    duration = (dt.datetime.strptime(maturity,'%m/%d/%y').date()-today).days
    return duration

def get_T2(): #days left before maturity(T1)
    shout = cal2.get_date()
    maturity = cal.get_date()
    durationT1 = (dt.datetime.strptime(maturity,'%m/%d/%y').date()-today).days
    durationshout = (dt.datetime.strptime(shout,'%m/%d/%y').date()-today).days
    return durationT1-durationshout

def show_2ndCal():
    # 2nd calender for users to pick a shout date
    #global shoutLab
    #shoutLab =Label(ws, text='   Shout Date:',font=15)
    #shoutLab.grid(row=7, column=2)
    global cal2
    cal2 = Calendar(ws,selectmode='day',#year=today.year,month=today.month,day=today.day,
                   mindate=today+dt.timedelta(1),maxdate=dt.datetime.strptime(cal.get_date(),'%m/%d/%y').date()-dt.timedelta(1),firstweekday='sunday',
                   selectbackground='white',selectforeground='green',
                   background='white',foreground='black', 
                   headersbackground='white', headersforeground='black',
                   othermonthbackground='white',othermonthforeground='#6B91C9',
                   cursor="hand2")
    global cal2but
    cal2but = Button(ws, text='Confirm Shout Date',command=show_T2)
    return cal2.grid(row=7,column=2,pady=20,padx=16), cal2but.grid(row=8,column=2,sticky="ew")

def show_T():
    tLabel = Label(ws,text=' '*50).grid(row=9,column=1)
    duration = get_T()
    tLabel = Label(ws, text=f'Time to Maturity: {round(duration/30,2)} months')
    
    choice = exo.get()
    if choice == 'Strike Reset Option':
        return tLabel.grid(row=9,column=1),show_2ndCal()
        
    return tLabel.grid(row=9,column=1)

def show_T2():
    global t2Label
    t2Label = Label(ws,text=' '*50)
    t2Label.grid(row=9,column=2)
    duration = get_T2()
    t2Label = Label(ws, text=f'{round(duration/30,2)} months before Maturity')
    
    return t2Label.grid(row=9,column=2)

def clear_(*args):
    for i in args:
        i.grid_remove()
    

# Get risk free rate from FRED
def get_rf():
    from fredapi import Fred
    fred = Fred(api_key='501431dcdd4d89dc0547ab37ac6ac7e2')
    data = fred.get_series('DGS1')
    rf  = fred.get_series('DGS1',today-dt.timedelta(days=5))[-1]/100 # The lastest risk free rate 
    return rf

# Vanilla call option using Black Scholes Formula 
# Use as reference option with known E[Z] for control variate method in getting an unbiased estimator
def euro_call(S,K,T,R,DIV,SIG):
    a=SIG*np.sqrt(T)
    d2=(np.log(S/K)+(R-DIV-(0.5*(SIG**2)))*T)/a
    d1=d2+a
    import scipy.stats
    NZ1=scipy.stats.norm.cdf(d1)
    NZ2=scipy.stats.norm.cdf(d2)
    return S*np.exp(-DIV*T)*NZ1-K*np.exp(-R*T)*NZ2

# Vanilla put option using Black Scholes Formula
def euro_put(S,K,T,R,DIV,SIG):
    a=SIG*np.sqrt(T)
    d2=(np.log(S/K)+(R-DIV-(0.5*(SIG**2)))*T)/a
    d1=d2+a
    import scipy.stats
    NZ1=scipy.stats.norm.cdf(-d1)
    NZ2=scipy.stats.norm.cdf(-d2)
    return K*np.exp(-R*T)*NZ2-S*np.exp(-DIV*T)*NZ1

def Asian_call(): # Asian option takes the average stock price in an interval as ST
    # To reduce simulation variance, control variate method will be used with vanilla call as Z
    
    # Parameters 
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    
    # Simulate parameter b in control variate method
    Nb = 10000
    A = np.zeros((Nb,1))
    for i in range(m):
        X = np.random.normal(0,1,size = (Nb,1))
        ST = S*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X)
        A = A + ST
        S = ST
    A_bar = A/m
    P = np.exp(-r*T)*np.maximum(A_bar-K,0)
    pcall = np.exp(-r*T)*np.maximum(S-K,0)
    b = np.std(P)*np.corrcoef(P[:,0],pcall[:,0])[0,1]/np.std(pcall)
    
    # Simulate original ybar
    S = stock_info()[0]
    A=np.zeros((N,1))
    for i in range(m):
        X = np.random.normal(0,1,size=(N,1))
        ST = S*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X)
        A = A + ST
        S = ST

    A_bar = A/m
   
    # Simulate vanilla call price
    pcall=np.exp(-r*T)*np.maximum(ST-K,0)
    # Apply control variate method Y-b(Z-E[Z])
    P_cv=np.exp(-r*T)*np.maximum(A_bar-K,0)-b*(pcall-euro_call(S,K,T,r,div,SIG))

    return np.mean(P_cv), np.std(P_cv)/np.sqrt(N)

def Asian_put():
    # To reduce simulation variance, control variate method will be used with vanilla put as Z
    
    # Parameters 
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    
    # Simulate parameter b in control variate method
    Nb = 10000
    A = np.zeros((Nb,1))
    for i in range(m):
        X = np.random.normal(0,1,size = (Nb,1))
        ST = S*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X)
        A = A + ST
        S = ST
    A_bar = A/m
    P = np.exp(-r*T)*np.maximum(K-A_bar,0)
    pput = np.exp(-r*T)*np.maximum(K-S,0)
    b = np.std(P)*np.corrcoef(P[:,0],pput[:,0])[0,1]/np.std(pput)
    
    # Simulate original ybar
    S = stock_info()[0]
    A=np.zeros((N,1))
    for i in range(m):
        X = np.random.normal(0,1,size=(N,1))
        ST = S*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X)
        A = A + ST
        S = ST

    A_bar = A/m
   
    # Simulate vanilla put price
    pput=np.exp(-r*T)*np.maximum(K-ST,0)
    # Apply control variate method Y-b(Z-E[Z])
    P_cv=np.exp(-r*T)*np.maximum(K-A_bar,0)-b*(pput-euro_put(S,K,T,r,div,SIG))

    return np.mean(P_cv), np.std(P_cv)/np.sqrt(N)

def Lookback_call(): # Lookback option takes the Best price in a time interval as ST
    # To reduce simulation variance, control variate method will be used with vanilla call as Z
    
     # Parameters 
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    
    # Simulate parameter b in control variate method
    Nb = 10000
    ST = S*np.ones((m+1,Nb))
    maxST = np.ones(Nb)*S
    X = np.random.normal(0,1,size = (m,Nb))
    for n in range(Nb):
        for i in range(m):
            ST[i+1,n] = ST[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X[i,n])
        maxST[n] = max(ST[:,n])
    
    P = np.exp(-r*T)*np.maximum(maxST-K,0)
    pcall = np.exp(-r*T)*np.maximum(ST[-1,]-K,0)
    b = np.std(P)*np.corrcoef(P,pcall)[0,1]/np.std(pcall)
    
    # Simulate original ybar
    S = stock_info()[0]
    ST = S*np.ones((m+1,N))
    maxST = np.ones(N)*S
    
    X = np.random.normal(0,1,size = (m,N))
    for n in range(N):
        for i in range(m):
            ST[i+1,n] = ST[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X[i,n])
        maxST[n] = max(ST[:,n])
   
   
    # Simulate vanilla call price
    pcall=np.exp(-r*T)*np.maximum(ST[-1]-K,0)
    # Apply control variate method Y-b(Z-E[Z])
    P_cv=np.exp(-r*T)*np.maximum(maxST-K,0)-b*(pcall-euro_call(S,K,T,r,div,SIG))

    return np.mean(P_cv), np.std(P_cv)/np.sqrt(N)

def Lookback_put():
    # To reduce simulation variance, control variate method will be used with vanilla put as Z
    
    # Parameters 
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    
    # Simulate parameter b in control variate method
    Nb = 10000
    ST = S*np.ones((m+1,Nb))
    minST = np.ones(Nb)*S
    X = np.random.normal(0,1,size = (m,Nb))
    for n in range(Nb):
        for i in range(m):
            ST[i+1,n] = ST[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X[i,n])
        minST[n] = min(ST[:,n])
    
    P = np.exp(-r*T)*np.maximum(K-minST,0)
    pput = np.exp(-r*T)*np.maximum(K-ST[-1,],0)
    b = np.std(P)*np.corrcoef(P,pput)[0,1]/np.std(pput)
    
    # Simulate original ybar
    S = stock_info()[0]
    ST = S*np.ones((m+1,N))
    minST = np.ones(N)*S
    
    X = np.random.normal(0,1,size = (m,N))
    for n in range(N):
        for i in range(m):
            ST[i+1,n] = ST[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X[i,n])
        minST[n] = min(ST[:,n])
   
   
    # Simulate vanilla put price
    pput=np.exp(-r*T)*np.maximum(K-ST[-1],0)
    # Apply control variate method Y-b(Z-E[Z])
    P_cv=np.exp(-r*T)*np.maximum(K-minST,0)-b*(pput-euro_put(S,K,T,r,div,SIG))

    return np.mean(P_cv), np.std(P_cv)/np.sqrt(N)

def Strike_Reset_call():
    #Reset Strike Option using conditional MC
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    T2 = get_T2()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    
    X=np.random.normal(0,1,size=(N,2))
    ST1=S*np.exp((r-SIG**2/2)*T+SIG*np.sqrt(T)*X[:,0])
    V=np.exp(-r*T)*(euro_call(ST1,np.minimum(K,ST1),T2,r,div,SIG))
    return np.mean(V),np.std(V)/np.sqrt(N)

def Strike_Reset_put():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    T2 = get_T2()/360 # days left before maturity
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    
    X=np.random.normal(0,1,size=(N,2))
    ST1=S*np.exp((r-SIG**2/2)*T+SIG*np.sqrt(T)*X[:,0])
    V=np.exp(-r*T)*(euro_put(ST1,np.maximum(K,ST1),T2,r,div,SIG))
    return np.mean(V),np.std(V)/np.sqrt(N)

def Binary_call():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())

    for i in range(m):
        X = np.random.normal(0,1,size=(N,1))
        ST = S*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X)
    V = np.where(ST>=K,1,0)
    return np.mean(V),np.std(V)/np.sqrt(N)

def Binary_put():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())

    for i in range(m):
        X = np.random.normal(0,1,size=(N,1))
        ST = S*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*X)
    V = np.where(ST<=K,1,0)
    return np.mean(V),np.std(V)/np.sqrt(N)


def Barrier_cond():
    knock = ['Knock In','Knock Out']
    global knockdropdown,in_out
    in_out = StringVar()
    in_out.set(knock[0])
    knockdropdown = OptionMenu(ws, in_out, *knock)
    knockdropdown.grid(row=1, column=2,sticky = 'ew')
    
    global baLab
    baLab = Label(ws, text='Barrier:',font=15,width = 7)
    baLab.grid(row=1, column=3,sticky='w')
    global Barrier
    B = IntVar(ws,value=stock_info()[0])
    Barrier = Entry(ws,textvariable = B,width = 7)
    Barrier.grid(row = 1,column = 4,sticky = 'ew')
    
    global baLink
    baLink = Label(ws, text="Learn Barrier Option", fg="blue", cursor="hand2")
    baLink.grid(row=2,column=4,sticky = 'ew')
    baLink.bind("<Button-1>", lambda event: webbrowser.open("https://en.wikipedia.org/wiki/Barrier_option"))
    

# All the barrier options have incorprated modeled extreme values
def Barrier_call_up_in():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    B = float(Barrier.get())
    
    U=np.random.uniform(0,1,size=(m,N)) # for modeling extreme values
    Z=np.random.normal(0,1,size=(m,N))
    S=S*np.ones((m+1,N))
    M=np.zeros((m,N)) # Extreme inbetween each interval
    P=np.zeros((N,1)) 

    for n in range(N):
        ind=0
        for i in range(m):
            S[i+1,n]=S[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*Z[i,n])
            logM=0.5*(np.log(S[i+1,n]*S[i,n])+np.sqrt((np.log(S[i+1,n]/S[i,n]))**2-2*SIG**2*h*np.log(U[i,n])))
            M[i,n]=np.exp(logM)
            if M[i,n]>B: 
                ind=1
                break
        P[n]=(np.maximum(S[-1,n]-K,0)*ind)*np.exp(-r*T)
    
    return np.mean(P),np.std(P)/np.sqrt(N)

def Barrier_call_down_out():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    B = float(Barrier.get())
    
    U=np.random.uniform(0,1,size=(m,N)) # for modeling extreme values
    Z=np.random.normal(0,1,size=(m,N))
    S=S*np.ones((m+1,N))
    M=np.zeros((m,N)) # Extreme values
    P=np.zeros((N,1)) 

    for n in range(N):
        ind=1
        for i in range(m):
            S[i+1,n]=S[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*Z[i,n])
            logM=0.5*(np.log(S[i+1,n]*S[i,n])+np.sqrt((np.log(S[i+1,n]/S[i,n]))**2-2*SIG**2*h*np.log(U[i,n])))
            M[i,n]=np.exp(logM)
            if M[i,n]<B: 
                ind=0
                break
        P[n]=(np.maximum(S[-1,n]-K,0)*ind)*np.exp(-r*T)
    
    return np.mean(P),np.std(P)/np.sqrt(N)


def Barrier_put_down_in():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    B = float(Barrier.get())
    
    U=np.random.uniform(0,1,size=(m,N)) # for modeling extreme values
    Z=np.random.normal(0,1,size=(m,N))
    S=S*np.ones((m+1,N))
    M=np.zeros((m,N)) # Extreme inbetween each interval
    P=np.zeros((N,1)) 

    for n in range(N):
        ind=0
        for i in range(m):
            S[i+1,n]=S[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*Z[i,n])
            logM=0.5*(np.log(S[i+1,n]*S[i,n])+np.sqrt((np.log(S[i+1,n]/S[i,n]))**2-2*SIG**2*h*np.log(U[i,n])))
            M[i,n]=np.exp(logM)
            if M[i,n]<B: 
                ind=1
                break
        P[n]=(np.maximum(K-S[-1,n],0)*ind)*np.exp(-r*T)
    
    return np.mean(P),np.std(P)/np.sqrt(N)

def Barrier_put_up_out():
    S = stock_info()[0]
    K = float(strikek.get())
    div = stock_info()[1]
    r = get_rf()
    T = get_T()/360
    SIG = comp_sigma()
    m = 10
    h = T/m
    N = int(simul.get())
    B = float(Barrier.get())
    
    U=np.random.uniform(0,1,size=(m,N)) # for modeling extreme values
    Z=np.random.normal(0,1,size=(m,N))
    S=S*np.ones((m+1,N))
    M=np.zeros((m,N)) # Extreme values
    P=np.zeros((N,1)) 

    for n in range(N):
        ind=1
        for i in range(m):
            S[i+1,n]=S[i,n]*np.exp((r-div-SIG**2/2)*h+SIG*np.sqrt(h)*Z[i,n])
            logM=0.5*(np.log(S[i+1,n]*S[i,n])+np.sqrt((np.log(S[i+1,n]/S[i,n]))**2-2*SIG**2*h*np.log(U[i,n])))
            M[i,n]=np.exp(logM)
            if M[i,n]>B: 
                ind=0
                break
        P[n]=(np.maximum(K-S[-1,n],0)*ind)*np.exp(-r*T)
    
    return np.mean(P),np.std(P)/np.sqrt(N)


def pick_exoption(choice):
    choice = exo.get()
    try: clear_(knockdropdown,baLab,Barrier,baLink)
    except: pass
    if choice != 'Strike Reset Option':
        try: 
            cal2.get_date()
            clear_(cal2,cal2but,t2Label)
        except: pass
        if choice == 'Lookback Option':
            
            link = Label(ws, text="Learn Lookback Option", fg="blue", cursor="hand2")
            #link.grid(row=1,column=2,sticky = 'ew')
            link.bind("<Button-1>", lambda event: webbrowser.open("https://en.wikipedia.org/wiki/Lookback_option"))
        if choice == 'Asian Option':
            link = Label(ws, text="Learn Asian Option", fg="blue", cursor="hand2")
            #link.grid(row=1,column=2,sticky = 'ew')
            link.bind("<Button-1>", lambda event: webbrowser.open("https://en.wikipedia.org/wiki/Asian_option"))
        if choice == 'Barrier Option':
            Barrier_cond()
            
        if choice == 'Binary Option':
            
            link = Label(ws, text="Learn Binary Option", fg="blue", cursor="hand2")
            link.grid(row=1,column=2,sticky = 'ew')
            link.bind("<Button-1>", lambda event: webbrowser.open("https://en.wikipedia.org/wiki/Binary_option"))
    else:#if choice == 'Strike Reset Option':
        
        link = Label(ws, text="Learn Strike Reset Option", fg="blue", cursor="hand2")
        #link.grid(row=1,column=2,sticky = 'ew')
        link.bind("<Button-1>", lambda event: webbrowser.open("https://www.sciencedirect.com/science/article/pii/S0895717711007187"))

    if choice != 'Barrier Option':
        return link.grid(row=1,column=2,sticky = 'ew')

def show_output():
    
    Labelp = Label(ws,text=' '*50).grid(row=12,column=1)
    Labelsd = Label(ws,text=' '*50).grid(row=13,column=1)
    
    if cp.get() == 1 and exo.get() == 'Asian Option':
        Labelp = Label(ws,text = f'Price: {round(Asian_call()[0],4)}')
        Labelsd = Label(ws,text = f'Standard Error: {round(Asian_call()[1],6)}')        
    
    if cp.get() == 2 and exo.get() == 'Asian Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Asian_put()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Asian_put()[1]}')
        
    if cp.get() == 1 and exo.get() == 'Lookback Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Lookback_call()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Lookback_call()[1]}')
        
    if cp.get() == 2 and exo.get() == 'Lookback Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Lookback_put()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Lookback_put()[1]}')
        
    if cp.get() == 1 and exo.get() == 'Strike Reset Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Strike_Reset_call()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Strike_Reset_call()[1]}')
        
    if cp.get() == 2 and exo.get() == 'Strike Reset Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Strike_Reset_put()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Strike_Reset_put()[1]}')
        
    if cp.get() == 1 and exo.get() == 'Binary Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Binary_call()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Binary_call()[1]}')
        
    if cp.get() == 2 and exo.get() == 'Binary Option':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Binary_put()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Binary_put()[1]}')
        
    if cp.get() == 1 and exo.get() == 'Barrier Option' and in_out.get() == 'Knock In':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Barrier_call_up_in()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Barrier_call_up_in()[1]}')
        
    if cp.get() == 1 and exo.get() == 'Barrier Option' and in_out.get() == 'Knock Out':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Barrier_call_down_out()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Barrier_call_down_out()[1]}')
    
    if cp.get() == 2 and exo.get() == 'Barrier Option' and in_out.get() == 'Knock In':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Barrier_put_down_in()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Barrier_put_down_in()[1]}')
        
    if cp.get() == 2 and exo.get() == 'Barrier Option' and in_out.get() == 'Knock Out':
        Labelp = Label(ws,text = f'Price: {"%0.4f" % Barrier_put_up_out()[0]}')
        Labelsd = Label(ws,text = f'Standard Error: {"%0.6f" % Barrier_put_up_out()[1]}')
        
    return Labelp.grid(row = 12, column=1,sticky='w'), Labelsd.grid(row = 13, column=1,sticky='w')

    
# Radiobutton to choose Call or Put
cp = IntVar()
cp.set(1)
Label(ws, text='   Call/Put:',font=15).grid(row=0, column=0,sticky='w')
cbutton = Radiobutton(ws, text="Call", variable=cp, value=1).grid(row=0, column=1, sticky='w')
qbutton = Radiobutton(ws, text="Put", variable=cp, value=2).grid(row=0, column=2, sticky='w')


# Dropdown list to choose Exotic option
Label(ws, text='   Exotic Option:',font=15).grid(row=1, column=0,sticky='w')
ex_option_list = ['Asian Option','Barrier Option','Binary Option','Lookback Option','Strike Reset Option']
exo = StringVar()
exo.set('Choose Option')
exdropdown = OptionMenu(ws, exo, *ex_option_list, command=pick_exoption).grid(row=1, column=1,sticky = 'ew')

# Enter Company Ticker
Label(ws, text='   Company Ticker:',font=15).grid(row=2, column=0,sticky='w')
com = StringVar(ws,value = 'AAPL')
comp = Entry(ws,textvariable = com)
comp.grid(row = 2,column = 1,sticky = 'ew')

Button(ws, text='Stock Price',command=show_S).grid(row=3,column=0,sticky='ew')
Button(ws, text='Dividend Yield',command=show_div).grid(row=3,column=1,sticky='ew')
Button(ws, text='5Y Volatility (Annulized)',command=show_sigma).grid(row=3,column=2,sticky='ew')

# Enter strike price 
Label(ws, text='   Strike Price:',font=15).grid(row=6, column=0,sticky='w')
strike = IntVar(ws,value=stock_info()[0])
strikek = Entry(ws,textvariable = strike)
strikek.grid(row = 6,column = 1,sticky="ew")


# Add a calender for users to pick a maturity date
Label(ws, text='   Maturity Date:',font=15).grid(row=7, column=0,sticky='w')
cal = Calendar(ws,selectmode='day',year=today.year,month=today.month,day=today.day,
               mindate=today+dt.timedelta(2),firstweekday='sunday',
               selectbackground='white',selectforeground='green',
               background='white',foreground='black', 
               headersbackground='white', headersforeground='black',
               othermonthbackground='white',othermonthforeground='#6B91C9',
               cursor="hand2")
cal.grid(row=7,column=1,pady=20)
Button(ws, text='Confirm Date',command=show_T).grid(row=8,column=1,sticky="ew")

# Enter Simulation Time
Label(ws, text='   Simulation Times:',font=15).grid(row=10, column=0,sticky='w')
simu = IntVar(ws,value=100)
simul = Entry(ws,textvariable = simu)
simul.grid(row = 10,column = 1,sticky="ew")

# Calculate expected payoff
Button(ws,text='Simulate Price',command = show_output).grid(row=11,column=1,pady=20)


# Show interface
ws.mainloop()


In [2]:
pip install fredapi

  Using cached fredapi-0.4.3-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.
